In [1]:
import os
import numpy as np
import pandas as pd
import re
import sys  
import matplotlib.pyplot as plt

In [2]:
text = pd.read_csv("./datasets/hongloumeng.csv")
import jieba
import jieba.analyse

vorc = [jieba.analyse.extract_tags(i, topK=1000) for i in text["text"]]
vorc = [" ".join(i) for i in vorc]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/j0/mlln6qvj2kl42rr7mg80lsxc0000gn/T/jieba.cache
Loading model cost 0.733 seconds.
Prefix dict has been built succesfully.


In [179]:
from  sklearn.feature_extraction.text import CountVectorizer
vertorizer = CountVectorizer(max_features=5000)
train_data_features = vertorizer.fit_transform(vorc)

In [180]:
train_data_features = train_data_features.toarray()

In [181]:
train_data_features.shape

(120, 5000)

In [182]:
labels = np.array([[0] * 80 + [1] * 40]).reshape(-1 ,1) # 目标值
labels.shape

(120, 1)

In [183]:
# 分层抽样

# X_train = train_data_features[[i for i in range(0, 120) if i % 4 != 0]]
# Y_train = labels[[i for i in range(0, 120) if i % 4 != 0]]

# X_test = train_data_features[[i for i in range(0, 120) if i % 4 == 0]]
# Y_test = labels[[i for i in range(0, 120) if i % 4 == 0]]
X_train, X_test, Y_train, Y_test = train_test_split(train_data_features, labels, test_size = 0.2)

In [184]:
# 分别取前80回中的60回，后40回中的30回进行训练

In [185]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [186]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [187]:
# 进行预测
y_pred = gnb.predict(X_test)

In [188]:
from sklearn.metrics import accuracy_score

In [189]:
accuracy_score(Y_test, y_pred)

0.7916666666666666

In [190]:
y_pred

array([1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [191]:
from sklearn.model_selection import cross_val_score
cross_val_score(gnb, X_train, Y_train, cv = 10, scoring="accuracy")

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

array([0.72727273, 0.8       , 0.9       , 0.8       , 0.9       ,
       0.77777778, 0.88888889, 0.88888889, 0.77777778, 0.66666667])